In [31]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
import plotly.express as px
import plotly.graph_objects as go
import altair as alt

In [32]:
employee_performance = pd.read_csv('../data_engineering/data_warehouse/report/employee_performance_kpi.csv')

unassigned_courses = pd.read_csv('../data_engineering/data_warehouse/report/unassigned_courses.csv')

In [33]:
employee_performance.columns

Index(['user_id', 'name', 'designation', 'course_id', 'course_title',
       'course_tag', 'course_duration', 'modules_completed', 'total_modules',
       'completion_percentage', 'course_score', 'performance_score'],
      dtype='object')

In [34]:
unassigned_courses.columns

Index(['user_id', 'name', 'designation', 'course_id', 'title', 'tag'], dtype='object')

HYPOTHESIS

Hypothesis: Employees with a higher completion percentage in courses achieve higher performance scores.

In [35]:
performance_summary = employee_performance.groupby(['user_id', 'name', 'designation']).agg({
    'completion_percentage': 'mean',
    'performance_score': 'mean'
}).reset_index()

# Create an interactive scatter plot
chart1 = alt.Chart(performance_summary).mark_circle(size=60).encode(
    x=alt.X('completion_percentage:Q', title='Average Completion Percentage (%)'),
    y=alt.Y('performance_score:Q', title='Average Performance Score'),
    tooltip=['completion_percentage', 'performance_score']
).properties(
    title='Course Completion Percentage vs. Performance Score',
    width=600,
    height=400
).interactive()

chart1.show()

alt.Chart(...)

Hypothesis: Courses with longer durations lead to greater performance score improvements.

In [36]:
# Aggregate by course duration
duration_performance = employee_performance.groupby('course_duration').agg({
    'performance_score': 'mean'
}).reset_index()

# Create scatter plot for course duration vs performance score
chart2 = alt.Chart(duration_performance).mark_circle(size=60).encode(
    x=alt.X('course_duration:Q', title='Course Duration (Minutes)'),
    y=alt.Y('performance_score:Q', title='Average Performance Score'),
    tooltip=['course_duration', 'performance_score']
).properties(
    title='Course Duration vs. Performance Score',
    width=600,
    height=400
).interactive()

chart2.show()


alt.Chart(...)

Hypothesis: There is a positive relationship between the number of modules completed in a course and the course score achieved by the employee.

In [37]:
# Aggregate data based on modules completed
modules_performance = employee_performance.groupby('modules_completed').agg({
    'course_score': 'mean'
}).reset_index()

# Create scatter plot for modules completed vs course score
chart4 = alt.Chart(modules_performance).mark_circle(size=60).encode(
    x=alt.X('modules_completed:Q', title='Modules Completed'),
    y=alt.Y('course_score:Q', title='Average Course Score'),
    tooltip=['modules_completed', 'course_score']
).properties(
    title='Modules Completed vs. Course Score',
    width=600,
    height=400
).interactive()

chart4.show()


alt.Chart(...)

In [38]:
employee_performance = pd.read_csv('../data_engineering/data_warehouse/report/employee_performance_kpi.csv')

# Aggregate data based on course tags and course scores
tag_score_summary = employee_performance.groupby('course_tag').agg({
    'course_score': 'mean'
}).reset_index()

# Define the threshold value for course score
threshold_value = 60

# Create a new column to identify whether the course score is above or below the threshold
tag_score_summary['course_tag_grouped'] = tag_score_summary['course_score'].apply(
    lambda score: 'Above 60' if score >= threshold_value else 'Below 60'
)

# Create a bar chart showing the average course score for each course tag
chart = alt.Chart(tag_score_summary).mark_bar().encode(
    x=alt.X('course_tag:O', title='Course Tag', sort='-y'),
    y=alt.Y('course_score:Q', title='Average Course Score'),
    color=alt.Color('course_tag_grouped:N', 
                    scale=alt.Scale(domain=['Above 60', 'Below 60'], 
                                    range=['#FFA500', '#FFDDC1']),
                    title='Tag Group'),
    tooltip=['course_tag', 'course_score']
).properties(
    title='Impact of Course Tags on Course Scores (Above and Below 60)',
    width=600,
    height=400
).interactive()

# Show the chart
chart.show()

alt.Chart(...)

FEATURE ENGINEERING

In [39]:
from sklearn.preprocessing import LabelEncoder

In [40]:
label_encoder = LabelEncoder()

# Perform label encoding on the 'course_tag' column
employee_performance['course_tag_encoded'] = label_encoder.fit_transform(employee_performance['course_tag'])

In [41]:
employee_performance['weighted_performance_score'] = (
    (employee_performance['course_score'] / (employee_performance['modules_completed'] + 1)) *
    employee_performance['course_tag_encoded']
)

In [42]:
display(employee_performance)
employee_performance.to_csv('employee_performance.csv')

,user_id,name,designation,course_id,course_title,course_tag,course_duration,modules_completed,total_modules,completion_percentage,course_score,performance_score,course_tag_encoded,weighted_performance_score
0,0564dc8a-2c53-44ac-b2ec-499755de840f,Shawna Flores,UI/UX Designer,4f3eb0d3-dd92-4afe-9c6c-cd1634770a52,Polarized static conglomeration,Mobile Development,117.0,4.0,7.0,57.142857,73.0,51.400000,5,73.000000
1,0564dc8a-2c53-44ac-b2ec-499755de840f,Shawna Flores,UI/UX Designer,5233bd5a-1fb4-4022-9ef4-9885ad9b99f4,Networked analyzing initiative,Software Testing,108.0,3.0,9.0,33.333333,60.0,51.400000,6,90.000000
2,0564dc8a-2c53-44ac-b2ec-499755de840f,Shawna Flores,UI/UX Designer,c388b418-ba14-4b9e-ae1f-b6ec6159ab06,Down-sized demand-driven paradigm,Generative AI,107.0,4.0,9.0,44.444444,26.0,51.400000,4,20.800000
3,07f6381b-edfe-4f3e-b485-0fc0d9fb6e97,Ryan Humphrey,AI Specialist,8e577a9b-05fd-42e0-b632-df1326c8f571,Optimized system-worthy concept,Mobile Development,159.0,2.0,6.0,33.333333,45.0,70.307692,5,75.000000
4,07f6381b-edfe-4f3e-b485-0fc0d9fb6e97,Ryan Humphrey,AI Specialist,a83b5073-16b7-4acd-b748-514eb1a25314,Public-key grid-enabled strategy,DevOps,106.0,2.0,7.0,28.571429,92.0,70.307692,3,92.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,f9d271d3-b328-4b2f-89bb-bd1b23ab7d32,James Wong,Cybersecurity Specialist,4f3eb0d3-dd92-4afe-9c6c-cd1634770a52,Polarized static conglomeration,Mobile Development,117.0,1.0,5.0,20.000000,24.0,47.384615,5,60.000000
96,f9d271d3-b328-4b2f-89bb-bd1b23ab7d32,James Wong,Cybersecurity Specialist,959f0001-a7c8-49f4-8bf7-173b2cb863b4,Sharable even-keeled flexibility,Web Development,150.0,4.0,8.0,50.000000,62.0,47.384615,8,99.200000
97,fc046613-47e8-427e-beb5-699680acb5b8,Elizabeth Brown,Data Engineer,f9623129-3998-4702-9949-2c9984428449,Automated 5thgeneration task-force,Mobile Development,175.0,5.0,7.0,71.428571,73.0,73.000000,5,60.833333
98,fd4a96d9-39b7-4d9b-a9b3-4fcb493e0e0b,Patricia Montoya,UI/UX Designer,e320cc11-248f-4fa2-9896-aaab2b4bc1ab,Re-contextualized composite system engine,DevOps,116.0,6.0,11.0,54.545455,21.0,21.000000,3,9.000000


In [43]:
employee_performance.columns

Index(['user_id', 'name', 'designation', 'course_id', 'course_title',
       'course_tag', 'course_duration', 'modules_completed', 'total_modules',
       'completion_percentage', 'course_score', 'performance_score',
       'course_tag_encoded', 'weighted_performance_score'],
      dtype='object')

In [45]:
employee_performance = employee_performance.drop(['user_id', 'name', 'designation', 'course_title', 'course_tag'], axis=1, errors='ignore')

In [48]:
employee_performance.columns

Index(['course_id', 'course_duration', 'modules_completed', 'total_modules',
       'completion_percentage', 'course_score', 'performance_score',
       'course_tag_encoded', 'weighted_performance_score'],
      dtype='object')

In [52]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Assuming you have already loaded your DataFrame
# employee_performance = pd.read_csv('path_to_your_file.csv')

# Clean up column names
employee_performance.columns = employee_performance.columns.str.strip()

# Check available columns
print("Available columns:", employee_performance.columns.tolist())

# Select features and target variable
X = employee_performance.drop(columns=['course_id', 'performance_score'])  
y = employee_performance['course_id']

# Apply label encoding to the course_tag_encoded column (if it's categorical)
label_encoder = LabelEncoder()
X['course_tag_encoded'] = label_encoder.fit_transform(X['course_tag_encoded'])

# Drop the original course_tag_encoded column after encoding
X = X.drop(columns=['course_tag_encoded'])

# Identify numerical features
numerical_features = X.columns.difference(['course_tag_encoded'])

# Define the preprocessing steps
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),  # Scaling numerical features
    ],
    remainder='passthrough'  # Keep remaining columns (if any)
)

# Create a pipeline with preprocessing and (optional) polynomial features
degree = 2  # Degree of polynomial features (if needed)
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('poly', PolynomialFeatures(degree=degree, include_bias=False)),  # Creating polynomial features
])

# Fit and transform the features
X_transformed = pipeline.fit_transform(X)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_transformed, y, test_size=0.2, random_state=42)

# Initialize the classifier
model = RandomForestClassifier(random_state=42)

# Train the model
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")
print(classification_report(y_test, y_pred))


Available columns: ['course_id', 'course_duration', 'modules_completed', 'total_modules', 'completion_percentage', 'course_score', 'performance_score', 'course_tag_encoded', 'weighted_performance_score']
Accuracy: 0.25
                                      precision    recall  f1-score   support

106b358b-973e-42b1-9154-94214dc7f604       0.00      0.00      0.00         0
26992488-3e60-4fa9-aa4b-6d5a4354e20b       1.00      1.00      1.00         1
2ff0eb16-b673-41dc-8f43-332b866d7b90       0.00      0.00      0.00         0
3a339568-c338-429e-8aef-293a3ea7f5bb       0.00      0.00      0.00         0
3c47df7c-1842-40b3-8899-68e3d95f41c8       0.00      0.00      0.00         1
3edbd79b-fa31-4627-a9e7-a788489c521a       0.00      0.00      0.00         1
4f3eb0d3-dd92-4afe-9c6c-cd1634770a52       0.67      1.00      0.80         2
5233bd5a-1fb4-4022-9ef4-9885ad9b99f4       0.00      0.00      0.00         0
695d5577-cbaa-4e8d-ad22-b8f8de421192       0.00      0.00      0.00         1


c:\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMet

In [50]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Step 1: Initialize the model
model = RandomForestRegressor(random_state=42)

# Step 2: Train the model
model.fit(X_train, y_train)

# Step 3: Make predictions on the test set
y_pred = model.predict(X_test)

# Step 4: Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R^2 Score: {r2}")


Mean Squared Error: 237.22709484296783
R^2 Score: 0.342852446627552


In [51]:
from sklearn.model_selection import GridSearchCV

# Define the parameter grid
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
}

# Initialize the GridSearchCV object
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, scoring='neg_mean_squared_error', n_jobs=-1)

# Fit the model
grid_search.fit(X_train, y_train)

# Best parameters
print(f"Best parameters: {grid_search.best_params_}")


Best parameters: {'max_depth': None, 'min_samples_split': 2, 'n_estimators': 200}


In [56]:
employees_df = pd.read_csv('../data_engineering/data_warehouse/report/employee_performance_kpi.csv')
employees_df.columns

Index(['user_id', 'name', 'designation', 'course_id', 'course_title',
       'course_tag', 'course_duration', 'modules_completed', 'total_modules',
       'completion_percentage', 'course_score', 'performance_score'],
      dtype='object')

In [57]:

df = employees_df[["course_id", 'course_duration', 'modules_completed', 'total_modules',
       'completion_percentage', 'course_score', 'performance_score']]
df.to_csv(r"C:\Users\SaiVikas\Desktop\final_project\data_science\temp_df.csv", index=Fale)

,course_id,course_duration,modules_completed,total_modules,completion_percentage,course_score,performance_score
0,4f3eb0d3-dd92-4afe-9c6c-cd1634770a52,117.0,4.0,7.0,57.142857,73.0,51.400000
1,5233bd5a-1fb4-4022-9ef4-9885ad9b99f4,108.0,3.0,9.0,33.333333,60.0,51.400000
2,c388b418-ba14-4b9e-ae1f-b6ec6159ab06,107.0,4.0,9.0,44.444444,26.0,51.400000
3,8e577a9b-05fd-42e0-b632-df1326c8f571,159.0,2.0,6.0,33.333333,45.0,70.307692
4,a83b5073-16b7-4acd-b748-514eb1a25314,106.0,2.0,7.0,28.571429,92.0,70.307692
...,...,...,...,...,...,...,...
95,4f3eb0d3-dd92-4afe-9c6c-cd1634770a52,117.0,1.0,5.0,20.000000,24.0,47.384615
96,959f0001-a7c8-49f4-8bf7-173b2cb863b4,150.0,4.0,8.0,50.000000,62.0,47.384615
97,f9623129-3998-4702-9949-2c9984428449,175.0,5.0,7.0,71.428571,73.0,73.000000
98,e320cc11-248f-4fa2-9896-aaab2b4bc1ab,116.0,6.0,11.0,54.545455,21.0,21.000000
